In [70]:
import pandas as pd
import os
import tarfile
from PIL import Image
import torchvision 
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset
from datetime import datetime

In [71]:
#Reading solar images
solar_Images=[f for f in os.listdir("img") if f.endswith(".jpg")]

#Loading images
transformed_Images=[]
for imageFilePath in solar_Images:
    image=Image.open("img\\"+imageFilePath)
    tensor_Transform=torchvision.transforms.ToTensor()
    transformed_Image=tensor_Transform(image)
    transformed_Images.append(transformed_Image)
    
images_Dataset=TensorDataset(transformed_Image)

In [72]:
#Reading flare dataset.
flareDataset=pd.read_csv('flareDataset.csv')
num_Rows, num_Columns=flareDataset.shape

#Setting everything to a universal time scale, with the 0 of time being the first image. 
initial_Time_Split=solar_Images[0].split("_")
date=initial_Time_Split[0]
time=initial_Time_Split[1]
s=""
s+=(f"{date[:4]}-{date[4:6]}-{date[6:]}")
s+=" "
s+=f"{time[:2]}:{time[2:4]}:{time[4:]}"
t0=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')

deltat_Images=[]

for imageName in solar_Images:
    s_split=imageName.split("_")
    s="" 
    date=s_split[0]
    time=s_split[1]
    s+=(f"{date[:4]}-{date[4:6]}-{date[6:]}")
    s+=" "
    s+=f"{time[:2]}:{time[2:4]}:{time[4:]}"
    t=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    deltat_Images.append((t-t0).total_seconds()/3600)
    
deltat_Flares=[]
for i in range(num_Rows):
    s_split=flareDataset.loc[i, "T_REC"].split(" ")
    s="" 
    date=s_split[0]
    time=s_split[1]
    s+=date
    s+=" "
    s+=time
    t=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    flareDataset.loc[i, "T_REC"]=(t-t0).total_seconds()/3600
    
for i in range(num_Rows):
    if int(flareDataset.loc[i,"T_REC"])<0 or int(flareDataset.loc[i, "FlareNumber"]) !=1:
        flareDataset.drop(i, inplace=True)
        
print(flareDataset)

      FlareNumber         T_REC  NOAA_AR  QUALITY  Longitude  Latitude  \
8494            1   1574.536111    11093        0         45        12   
8495            1   3255.736111    11112     1024         15        19   
8496            1   6134.136111    11158        0         11        19   
8497            1   6157.936111    11158        0          3        20   
8498            1   6166.336111    11158        0         14        21   
...           ...           ...      ...      ...        ...       ...   
8869            1  63707.736111    12673        0         58         9   
8870            1  63716.136111    12673        0         58         9   
8871            1  63720.936111    12673        0         70         9   
8872            1  63727.536111    12673        0         70         9   
8873            1  63738.736111    12673        0         70         9   

       TOTUSJH        TOTBSQ        TOTPOT       TOTUSJZ  ...  MEANGBT  \
8494   956.465  2.291500e+10  2.26499

In [73]:

def attachActualValues():
    pointer1 = 0;
    pointer2 = 0;
    solarFlareDates = flareDataset.T_REC.array
    newData = []



    for tensor in transformed_Images:
        if(deltat_Images[pointer1] > solarFlareDates[pointer2]):
            pointer2 += 1

        dateOfImage = deltat_Images[pointer1]
        dateOfNextSolarFlare = solarFlareDates[pointer2]
        dateToNextFlare = dateOfNextSolarFlare-dateOfImage

        newData.append((tensor, dateToNextFlare))
        pointer1 += 1

    return newData

actualValues = attachActualValues()
print(actualValues)


[(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]), 1574.536111111111), (tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0.

In [74]:
#Residual neural network for regression.

weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1
resnet=torchvision.models.resnet18(weights)

#Replacing classification layer with regression layer.
resnet.fc=nn.Linear(512, 1, bias=False)




C:\Users\Jacob Kogut\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
